Este codigo permite a extracao dos dados propierties BR, transformacao e envio dos dados por email.
O codigo se pode planejar com airflow ou em algum servidor na nuvem com crostab executando um ssh que execute o python com os parametros de data. Na bigquery gratuita esta limitado esse uso por isso so fiz os codigos que serao implementados para automatizacao. 

Pode ser planejado diariamente especificando os parametros de ini e fim.

A extracao permite reprocessamento pelo periodo indicado 
A transformacao gera a fato mensal pelo periodo indicado
O email envia a media ate o mes executado ao cliente

In [29]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
#bigquery-public-data.samples.natality

Authenticated


extracao dos dados e limpeza=

* se colocou um filtro de tempo para planejar esta execucao enviando por parametros data de inicio e fim se precisar.
* Se incorporou uma sentenca de delete para disponibilizar o reprocessamento mas ficou desabilitado porque a versao gratuita de bigquery nao possibilita isso. Mas coloquei substituir para conseguir reprocessar.
* a extracao filtra os dados por cidade de sao paulo, apartamento e por periodo. Assegura que o price seja existente para usar no calculo da media



In [ ]:

from google.cloud import bigquery 
from datetime import datetime
import pandas as pd
from sqlalchemy import create_engine
import io
from google.colab import files
import pandas_gbq


#confiGUration projeto 
project_id = "teste-306620"
client = bigquery.Client(project=project_id)
dataset_id="properati-data-public.properties_br"
table_id = "TESTEUS.STG_PROPERTIES_BR"
table_destino = """`teste-306620.TESTEUS.STG_PROPERTIES_BR`"""
# fim 

#DATA DE PROCESSAMENTO #
INI_DATE="2017/01/01"
FIM_DATE="2017/12/31"
#FIM DATA DE PROCESSAMENTO #

#FIM_DATE_DT = datetime.strptime(FIM_DATE, '%Y/%m/%d').date()
tables = client.list_tables(dataset_id)  # Make an API request.

#ADQUIRIR OS DADOS DA CIDADE DO SAO PAULO DOS APARTAMENTOS #
count=0
#sql_delete=""" delete from """+table_destino+"""  where  PARSE_DATE('%Y-%m-%d', substr(cast(created_on as string),1,10)) between PARSE_DATE('%Y/%m/%d', '"""+INI_DATE+"""')and PARSE_DATE('%Y/%m/%d', '"""+FIM_DATE+"""') """
#df = pandas_gbq.read_gbq(sql_delete, project_id=project_id)  

for table in tables:
 
    sql="""
    SELECT id	,created_on, operation	, property_type	, place_name,place_with_parent_names	,state_name,cast(price as FLOAT64)	price	
    FROM  `"""+"""{}.{}.{}""".format(table.project, table.dataset_id, table.table_id)+ """`
        where  
        created_on between PARSE_DATE('%Y/%m/%d', '"""+INI_DATE+"""')and PARSE_DATE('%Y/%m/%d', '"""+FIM_DATE+"""') 
        and state_name = 'São Paulo'  
        AND upper(property_type) like '%APART%' 
        and price is not null and price>0
        and  place_with_parent_names  like '%|São Paulo|São Paulo|%'  """
    df = pandas_gbq.read_gbq(sql, project_id=project_id)
    if  count ==0:
        pandas_gbq.to_gbq(df, table_id, project_id=project_id,if_exists='replace')
    else:
        pandas_gbq.to_gbq(df, table_id, project_id=project_id,if_exists='append')
    count=1



Transformacao Load

* se realizou um calculo mensal da media dos apartamentos para o aluguel e para a venda
* permite reprocessamento de acordo as datas inicio e fim. Mas o delete tem uma limitacao pelo uso de bigquery gratuito mas se resolveu fazendo um replace dos dados da tabela

In [31]:


from google.cloud import bigquery 
from datetime import datetime
import pandas as pd
from sqlalchemy import create_engine
import io
from google.colab import files
import pandas_gbq
#confiGUration projeto 
project_id = "teste-306620"
client = bigquery.Client(project=project_id)
dataset_id="properati-data-public.properties_br"
table_id = "TESTEUS.STG_PROPERTIES_BR"
table_destino = "TESTEUS.FT_PROPERTIES_BR"

#DATA DE PROCESSAMENTO #
INI_DATE="2017/01/01"
FIM_DATE="2017/12/31"
INI=INI_DATE[0:4]+INI_DATE[5:7]
FIM=FIM_DATE[0:4]+FIM_DATE[5:7]

#FIM DATA DE PROCESSAMENTO #

#sql_delete="""delete from `teste-306620.TESTEUS.FT_PROPERTIES_BR` WHERE MES_ID BETWEEN  cast('"""+INI+"""'as int64) AND cast('"""+FIM+"""'as int64) """
print(sql_delete)
#pandas_gbq.read_gbq(sql_delete, project_id=project_id)  

sql="""select operation, EXTRACT(YEAR FROM CREATED_ON)*100+ EXTRACT(MONTH FROM CREATED_ON) mes_id, round(sum(price)/count(1),2) media_price 
from `teste-306620.TESTEUS.STG_PROPERTIES_BR` 
where  PARSE_DATE('%Y-%m-%d', substr(cast(created_on as string),1,10)) between PARSE_DATE('%Y/%m/%d', '"""+INI_DATE+"""')and PARSE_DATE('%Y/%m/%d', '"""+FIM_DATE+"""') 
group by operation , EXTRACT(YEAR FROM CREATED_ON)*100+ EXTRACT(MONTH FROM CREATED_ON)"""
df = pandas_gbq.read_gbq(sql, project_id=project_id)
pandas_gbq.to_gbq(df, table_destino, project_id=project_id,if_exists='replace')



delete from `teste-306620.TESTEUS.FT_PROPERTIES_BR` WHERE MES_ID BETWEEN  cast('201701'as int64) AND cast('201712'as int64) 


Downloading: 100%|██████████| 24/24 [00:00<00:00, 88.53rows/s]
1it [00:02,  2.37s/it]


enviar o email 

Se realizou um email para enviar os resultados ao ultimo mes processado. Discriminado por aluguel e venda dos apartamentos


In [36]:
# função enviar_email
import smtplib
import email.message

def enviar_email(resumo_loja, loja):

  server = smtplib.SMTP('smtp.gmail.com:587')  
  email_content = f'''
  <p>Estimado,
  Segue o preco medo dos apartamentos na Ciudade de Sao paulo para aluguel e venda
  </p>
  {resumo_loja.to_html()}
  <p>Atenciosamente, Cynthia Mendoza </p>'''
  
  msg = email.message.Message()
  msg['Subject'] = f'Properties BR Mensal'
  
  
  msg['From'] = 'piruvarideveloper1@gmail.com'
  msg['To'] = 'piruvarideveloper@gmail.com'
  password = 'supersenha'
  msg.add_header('Content-Type', 'text/html')
  msg.set_payload(email_content)
  
  s = smtplib.SMTP('smtp.gmail.com: 587')
  s.starttls()
  # Login Credentials for sending the mail
  s.login(msg['From'], password)
  s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))

In [38]:

sql="""select operation ,  round(avg(media_price),2) media_anual
from `teste-306620.TESTEUS.FT_PROPERTIES_BR` 
group by operation
"""
df = pandas_gbq.read_gbq(sql, project_id=project_id)

enviar_email(df, 'Preco medio dos apartamentos da Cidade de Sao Paulo')

Downloading: 100%|██████████| 2/2 [00:00<00:00, 11.98rows/s]
